# Application Optimization

## Description

The goal is to optimize the decision of applying to something like universities, positions, jobs etc. In these situations, there are multiple options, but in the end only one can be chosen. Thus, in the end, regardless of which universities, positions, or jobs that someone had applied to, the only one that matters is the one which the person chooses to accept. Hence, it is a different class of problem than the ones commonly considered, such as stock selection.

## Assumptions
The following assumptions are made regarding the nature of the problem:
- The preference of the person for the position can be captured adequately by reward or utility. That is, the preference of the person can be reduced to a number and its associated operations. To this end, the most fitting way of setting the reward may be to associate the positions with monetary values.
- The acceptence of an application to a position is independent from the other positions. That is, getting accepted to a position depends only on the position and the application. This may or may not be a reasonable assumption depending on context. However, if there are no mutual relations or sharing of informations between the reviewers for a given position, then assuming independence should not cause significant issue.
- The cost of applying to a position is negligible comparing to its reward. For most applications, this would be a reasonable assumption, since filling out forms are not comparable to getting a degree from a university for instance. In any case, the reward can be adjusted to reflect the cost of applications if needed.

## Derivation

### Problem Setup

Let $N$ be the number of candidate positions that can be applied to.

Let $M$ be the number of applications that can be submitted.

Let $L \in \mathbb{R}_+^N$ be the reward vector associated with the $N$ candidates sorted ascending.

Let $X \in \{0, 1\}^N$ be the decision vector representing to which positions are the applications submitted.

Let $T \in \{0, 1\}^N$ be the state representation indicating which positions would be willing to accept if applied to.

Let $P \in [0, 1]^N$ be the probabibilities of being accepted by a position if applied.


### Decision Rule and Rewards

The decision rule under consideration is to apply to the positions and then choose the best one accepted. The decision itself involved a fixed vector of positions to apply to. The choice is deterministic, so the reward function is:$$R(X, T) = max(0, L_1X_1T_1, L_2X_2T_2,...,L_NX_NT_N)$$

Then, considering the prior probabilities of being accepted by the positions, the Bayes reward function is:
$$\mathbb{E}[R(X, T)]=\mathbb{E}[max(0, L_1X_1T_1, L_2X_2T_2,...,L_NX_NT_N)]=\mathbb{E}[\mathbb{E}[max(0, L_1X_1T_1, L_2X_2T_2,...,L_NX_NT_N)| I]]$$

Where $I$ is the RV indicating the index of the last term that equals the max value.

Evaluating the expectation:
$$\mathbb{E}[\mathbb{E}[max(0, L_1X_1T_1, L_2X_2T_2,...,L_NX_NT_N)| I]]=\sum_{i=1}^N X_iT_iL_iP(I=i)$$
Since the $L$ is ordered ascending, the ith term is the last term identical to the max iff the terms after it are 0, and the term itself is non-zero. Thus, $$P(I=i) = X_iP_i P(L_jX_jT_J = 0 : j > i)$$

Since the acceptances are independent,
$$P(L_jX_jT_j = 0 : j > i) = \prod_{j>i} P(X_jT_j = 0)=\prod_{j>i} (1-X_j)+X_j(1-P_j)=\prod_{j>i} 1-X_jP_j$$

Thus,
$$\mathbb{E}[\mathbb{E}[max(L_0X_0T_0, L_1X_1T_1, L_2X_2T_2,...,L_NX_NT_N)| I]]= \sum_{i=1}^N [L_iX_iP_i\prod_{j>i} (1-X_jP_j)]$$

## The Model

### Formulation

Using the Bayes reward as the objective function, and noting that there can be at most $M$ applications, the description of the problem becomes:
$$maximize \sum_{i=1}^N [L_iX_iP_i\prod_{j>i} (1-X_jP_j)]$$

subject to:
$$ \sum_{i=1}^N X_i = M$$

### Code

In [16]:
import pyomo.environ as pyo

model = pyo.AbstractModel()
model.m = pyo.Param(within=pyo.PositiveIntegers, default=3)
model.n = pyo.Param(within=pyo.PositiveIntegers, default=3)
model.N = pyo.RangeSet(1, model.n)
model.L = pyo.Param(model.N, within=pyo.PositiveReals)
model.P = pyo.Param(model.N, within=pyo.PercentFraction)
model.x = pyo.Var(model.N, within=pyo.Binary)


def rest_zero(current, model):
    result = 1
    for j in range(current + 1, model.n.value + 1):
        result = (1 - model.x[j] * model.P[j]) * result
    return result


def reward_(model):
    return sum([model.L[i] * model.x[i] * model.P[i] * rest_zero(i, model) for i in model.N])


model.reward = pyo.Objective(rule=reward_, sense=pyo.maximize)


def limit_(model):
    return pyo.summation(model.x) == model.m


model.limit = pyo.Constraint(rule=limit_)

Instantiating with test data:

In [21]:
data = {None: {
        "n": {None: 6},
        "m": {None: 2},
        "L": {1: 10, 2: 13, 3: 15, 4: 50, 5: 100, 6: 200},
        "P": {1: 0.9, 2: 0.8, 3: 0.8, 4: 0.4, 5: 0.1, 6: 0.05}
    }
}

instance = model.create_instance(data)
instance.pprint()

1 RangeSet Declarations
    N : Dimen=1, Size=6, Bounds=(1, 6)
        Key  : Finite : Members
        None :   True :   [1:6]

4 Param Declarations
    L : Size=6, Index=N, Domain=PositiveReals, Default=None, Mutable=False
        Key : Value
          1 :    10
          2 :    13
          3 :    15
          4 :    50
          5 :   100
          6 :   200
    P : Size=6, Index=N, Domain=PercentFraction, Default=None, Mutable=False
        Key : Value
          1 :   0.9
          2 :   0.8
          3 :   0.8
          4 :   0.4
          5 :   0.1
          6 :  0.05
    m : Size=1, Index=None, Domain=PositiveIntegers, Default=3, Mutable=False
        Key  : Value
        None :     2
    n : Size=1, Index=None, Domain=PositiveIntegers, Default=3, Mutable=False
        Key  : Value
        None :     6

1 Var Declarations
    x : Size=6, Index=N
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :     1 : False :  True : Binary
          2 :

In [22]:
opt = pyo.SolverFactory("couenne")
opt.solve(instance)
instance.x.pprint()
print("Reward: " + str(instance.reward()))

x : Size=6, Index=N
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   1.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary
      6 :     0 :   1.0 :     1 : False : False : Binary
Reward: 29.0


## Discussion
The problem is integer non-linear, and does not fit into quadratic or bi-linear programming. Due to the expectation of the maximum, the objective function would contain the products of the variable with each other. Since all of the variables are binary, it is not convex. Thus, the decision version of the problem for the feasibility of a lower bound is likely to be NP-hard. It is obvious that there are $2^N$ states in the problem, so an enumeration algorithm checking every state would have the complexity $O(2^N)$ in the number of positions to consider. Given that a person would likely not apply to a large number of position (say beyond 50), it would be sufficient for most uses. However, there may be other formulations or techniques that admits of a faster algorithm. 

Since the problem of applying to positions, universities, or jobs etc are commonplace in daily life, it is a very practical application. In the context of making decisions with limited opportunities to apply to positions, this can be useful as a tool to guide the selection. Additionaly, since the state is encoded via a vector of bernoulli RVs, more sophisticated domain-based hierarchical models can be built to assign the probabilities for the bernoulli variables. Then, the current solution can be part of a larger model that can lead to better decisions. Finally, there are cases where the independence assumption for the positions do not hold. In these cases, the current formulation may provide some guidance, but more sophisticated model allowing for dependence relations between acceptence to positions would be required.